In [6]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.utils import shuffle
from sklearn.pipeline import Pipeline
import os

import pickle

In [7]:
from sklearn.externals import joblib

In [8]:
f = lambda f_name: os.path.realpath(os.path.join(os.getcwd(), f_name)).replace('\\', '/')

In [9]:
train = pd.read_csv(f("final.csv"), header = 0, sep="\t")

In [13]:
train_text = train['review_text']
train_label = train['label']

In [5]:
#function that returns trained classifier object 
def my_classifier(text, label):
    #prod_sentim_train = pd.read_csv("products_sentiment_train.tsv", names = ["text", "label"], header = 0, sep="\t")
    #zero_test_ind = prod_sentim_train[prod_sentim_train["label"] == 0]
    #new_prod_sentim_train = shuffle(prod_sentim_train.append(prod_sentim_train.ix[list(zero_test_ind.index[:500])]), random_state=10)

    def text_classifier(vectorizer, transformer, classifier):
        return Pipeline(
                [("vectorizer", vectorizer),
                ("transformer", transformer),
                ("classifier", classifier)]
            )

    Lin_SVC = text_classifier(vectorizer=CountVectorizer(min_df=1, ngram_range=(1, 2), max_df=0.85, stop_words=None), 
                    transformer=TfidfTransformer(norm = 'l2', smooth_idf = True, use_idf = True, sublinear_tf = True), 
                    classifier=LinearSVC(max_iter=400, loss='squared_hinge', C= 1.0, tol=1e-05, random_state=1))

    Lin_SVC.fit(text, label)

    return Lin_SVC    

In [21]:
cls = my_classifier(train_text, train_label)
print(cls)

In [7]:
#pickle lib to save to file trained classifier object
#write to file serilized object to file; protocol=3 for python 3
with open(f('dumped_lin_SVC_classifier.pkl'), 'wb') as fl:
    pickle.dump(cls, fl)

In [20]:
model = joblib.load(f("dumped_lin_SVC_classifier.pkl"))

In [22]:
print(model)

In [23]:
''' check check whether the text is written with symbols in ascii '''
for i in range(10):
    print((lambda s: all(ord(c) < 128 for c in s))(train_text.iloc[i]))
#return False because reviews are in сyrillic alphabet    